### calculated all the benchmarking metrics

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import anndata as ad
import scanpy as sc

import sys
sys.path.append("./")
import match
import metrics
import utils
from scipy.io import mmread
import os

sys.executable

'/home/bkzhu/python/miniconda3/envs/super_mario_testing/bin/python3'

In [2]:
meta = pd.read_csv('/atac_bench_nrz/retina/data/meta_20k.csv')
meta

,X,orig.ident,nCount_RNA,nFeature_RNA,nCount_ATAC,nFeature_ATAC,annotation
0,2_TTTCTTGCACGGTACT-1,2,10065,3906,9999.971,15090,GABAamacrine
1,6_TGCGATTAGCCTTAAA-1,6,1455,1065,9999.954,4610,GABAamacrine
2,3_CTCGCTAGTCCCGAAG-1,3,1574,983,10000.020,8736,Rod
3,1_TCGTTATTCAAGTGTC-1,1,6082,2764,10000.021,12601,Cone
4,4_GAGGACTAGTATTGCA-1,4,6819,2698,9999.984,15899,Mullerglia
...,...,...,...,...,...,...,...
19995,2_ATGGACAAGCTACTGG-1,2,6598,2696,10000.057,13472,Rod
19996,2_ATTGCGCCATCATGGC-1,2,12312,4125,10000.020,16558,ONconebipolar
19997,3_ACGCCTAAGGACACTT-1,3,2909,1564,10000.015,7784,Rod
19998,3_ACCCTGTTCGGTTAGT-1,3,2128,1260,9999.977,6840,Rod


In [3]:
## glue
glx = pd.read_csv("/atac_bench_nrz/retina/glue/sub_rna_embed.csv")
gly = pd.read_csv("/atac_bench_nrz/retina/glue/sub_atac_embed.csv")
gl_dist = utils.cdist_correlation(gly.to_numpy(), glx.to_numpy())
gl_full_match, gl_scores = metrics.get_knn_matching(gl_dist)
glmatch = [np.arange(glx.shape[0]),gl_full_match,gl_scores]

annt = meta['annotation'].astype(str).to_numpy()
order = (2, 1)
acc_ann_gl = metrics.get_matching_acc(
        matching=glmatch, 
        labels1=annt, 
        labels2=annt,
        order = order
    )
acc_ann_gl # 

0.9724910850738665

In [5]:
## scj
scjx = pd.read_csv("/atac_bench_nrz/retina/scj/sub_rna_embed.csv")
scjy = pd.read_csv("/atac_bench_nrz/retina/scj/sub_atac_embed.csv")
scj_dist = utils.cdist_correlation(scjy.to_numpy(), scjx.to_numpy())
scj_full_match, scj_scores = metrics.get_knn_matching(scj_dist)
scjmatch = [np.arange(scjx.shape[0]),scj_full_match,scj_scores]

annt = meta['annotation'].astype(str).to_numpy()
order = (2, 1)
acc_ann_scj = metrics.get_matching_acc(
        matching=scjmatch, 
        labels1=annt, 
        labels2=annt,
        order = order
    )
acc_ann_scj # 

0.9553627760252366

In [6]:
## maestro
msx = pd.read_csv("/atac_bench_nrz/retina/ms/sub_rna_embed.csv")
msy = pd.read_csv("/atac_bench_nrz/retina/ms/sub_atac_embed.csv")
ms_dist = utils.cdist_correlation(msy.to_numpy(), msx.to_numpy())
ms_full_match, ms_scores = metrics.get_knn_matching(ms_dist)
msmatch = [np.arange(msx.shape[0]),ms_full_match,ms_scores]

annt = meta['annotation'].astype(str).to_numpy()
order = (2, 1)
acc_ann_ms = metrics.get_matching_acc(
        matching=msmatch, 
        labels1=annt, 
        labels2=annt,
        order = order
    )
acc_ann_ms # 

0.8522072936660269

In [7]:
mf = pd.read_csv("/atac_bench_nrz/retina/mf/full_idx_21_test.csv")
mf_meta = pd.read_csv("/atac_bench_nrz/retina/data/meta_20k.csv")
mfmatch = [mf['idx1'].tolist(),mf['idx2'].tolist(),mf['score'].tolist()]

annt = mf_meta['annotation'].astype(str).to_numpy()
order = (2, 1)
acc_ann_mf = metrics.get_matching_acc(
        matching=mfmatch, 
        labels1=annt, 
        labels2=annt,
        order = order
    )
acc_ann_mf

0.963

In [8]:
ann_listlv1 = []
ann_listlv1.extend([acc_ann_mf, acc_ann_gl, acc_ann_scj, acc_ann_ms])
ann_listlv1

[0.963, 0.9724910850738665, 0.9553627760252366, 0.8522072936660269]

In [9]:
mfx = pd.read_csv("/atac_bench_nrz/retina/mf/full_embed_x0_test.csv")
mfy = pd.read_csv("/atac_bench_nrz/retina/mf/full_embed_y0_test.csv")

mf_dist = utils.cdist_correlation(mfy.to_numpy(), mfx.to_numpy())

In [10]:
mf_fos = metrics.get_foscttm(mf_dist)
gl_fos = metrics.get_foscttm(gl_dist)
scj_fos = metrics.get_foscttm(scj_dist)
ms_fos = metrics.get_foscttm(ms_dist)

foscttm_list = []
foscttm_list.extend([mf_fos, gl_fos, scj_fos, ms_fos])
foscttm_list

[0.07444833499999999, 0.1154081975, 0.1307757725, 0.1862192675]

In [11]:
slt_f1 = []
ari_f1 = []

mf_int = pd.read_csv("/atac_bench_nrz/retina/mf/metrics.csv")
ms_int = pd.read_csv("/atac_bench_nrz/retina/ms/metrics.csv")
gl_int = pd.read_csv("/atac_bench_nrz/retina/glue/metrics.csv")
scj_int = pd.read_csv("/atac_bench_nrz/retina/scj/metrics.csv")

slt_f1.extend([mf_int.loc[0,'slt_f1'], gl_int.loc[0,'slt_f1'], scj_int.loc[0,'slt_f1'], ms_int.loc[0,'slt_f1']])
ari_f1.extend([mf_int.loc[0,'ari_f1'], gl_int.loc[0,'ari_f1'], scj_int.loc[0,'ari_f1'], ms_int.loc[0,'ari_f1']])

slt_f1

[0.5607986277518464,
 0.5930791321843494,
 0.5987924473501434,
 0.5746222885018761]

In [12]:
m = ["mf","gl","scj","ms"]
minfo = m

data = {'method':minfo,'slt_f1': slt_f1, 'ari_f1':ari_f1,
       'ann1':ann_listlv1, 'foscttm':foscttm_list}

matching_result = pd.DataFrame(data)
matching_result

,method,slt_f1,ari_f1,ann1,foscttm
0,mf,0.560799,0.565910,0.963000,0.074448
1,gl,0.593079,0.565955,0.972491,0.115408
2,scj,0.598792,0.539765,0.955363,0.130776
3,ms,0.574622,0.577142,0.852207,0.186219


In [20]:
matching_result.to_csv("/atac_bench_nrz/retina/metric_results.csv")

In [13]:
# knn search tmp
k = 100
knn_match_mf = metrics.get_knn_alignment_score(
    dist=mf_dist,
    k_max=k
)

knn_match_gl = metrics.get_knn_alignment_score(
    dist=gl_dist,
    k_max=k
)

knn_match_scj = metrics.get_knn_alignment_score(
    dist=scj_dist,
    k_max=k
)

knn_match_ms = metrics.get_knn_alignment_score(
    dist=ms_dist,
    k_max=k
)


In [14]:
data = {'mf': knn_match_mf,
        'gl':knn_match_gl,
        'scj':knn_match_scj,
        'ms':knn_match_ms
       }
knn_tmp = pd.DataFrame(data)
knn_tmp

,mf,gl,scj,ms
0,0.01415,0.01595,0.00270,0.00280
1,0.02175,0.02450,0.00465,0.00555
2,0.02820,0.03120,0.00695,0.00740
3,0.03300,0.03675,0.00850,0.00920
4,0.03730,0.04145,0.01020,0.01095
...,...,...,...,...
95,0.22065,0.23460,0.09330,0.12585
96,0.22215,0.23575,0.09380,0.12685
97,0.22330,0.23685,0.09425,0.12765
98,0.22435,0.23850,0.09470,0.12895


In [18]:
import numpy as np

knn_tmp = []
knn_tmp.extend([knn_match_mf.tolist(), knn_match_gl.tolist(),
                knn_match_scj.tolist(), knn_match_ms.tolist()])
m = np.array(["mf", "gl","scj","ms"])
minfo = np.repeat(m, [100,100,100,100], axis=0).tolist()
minfo2 = minfo #* 5
knn = [item for sublist in knn_tmp for item in sublist]
step = [i for i in range(100)]*4 #*5
data = {'method':minfo2,'knn_tmp': knn, 'step':step }

knn_result = pd.DataFrame(data)
knn_result

,method,knn_tmp,step
0,mf,0.01415,0
1,mf,0.02175,1
2,mf,0.02820,2
3,mf,0.03300,3
4,mf,0.03730,4
...,...,...,...
395,ms,0.12585,95
396,ms,0.12685,96
397,ms,0.12765,97
398,ms,0.12895,98


In [19]:
# save the results as csvs
knn_result.to_csv("/atac_bench_nrz/retina/knntmp.csv")